# WEEK 3  IBM CAPSTONE 

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import requests 
from bs4 import BeautifulSoup 


print('Libraries imported.')

Libraries imported.


In [2]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(URL) 
  
soup = BeautifulSoup(r.content, 'html.parser') 

#print(soup.prettify()) 

print('Page Scrapped.')

Page Scrapped.


In [3]:
tables = soup.find('table', attrs={'class':'wikitable'})

table_rows = tables.find_all('tr')
        
l = []
for tr in table_rows[2:]:
    td = tr.find_all('td')
    row = [tr.text.replace('\n', '') for tr in td]
    l.append(row)

df = pd.DataFrame(l, columns=['PostalCode', 'Borough', 'Neighborhood'])
df

,PostalCode,Borough,Neighborhood
0,M2A,Not assigned,
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,Regent Park / Harbourfront
4,M6A,North York,Lawrence Manor / Lawrence Heights
...,...,...,...
174,M5Z,Not assigned,
175,M6Z,Not assigned,
176,M7Z,Not assigned,
177,M8Z,Etobicoke,Mimico NW / The Queensway West / South of Bloo...


In [4]:
#Cleaning the data Removing Not assigned
df_Toronto = df[df.Borough != 'Not assigned']

In [5]:
df_Toronto

,PostalCode,Borough,Neighborhood
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,Regent Park / Harbourfront
4,M6A,North York,Lawrence Manor / Lawrence Heights
5,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
...,...,...,...
159,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North
164,M4Y,Downtown Toronto,Church and Wellesley
167,M7Y,East Toronto,Business reply mail Processing CentrE
168,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...


In [6]:
toronto_geocsv = 'https://cocl.us/Geospatial_data'

geocsv_data = pd.read_csv(toronto_geocsv)
geocsv_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
#Combining Postal CODE of geocsv_data to df_Toronto
toronto_neighbors = df_Toronto.join(geocsv_data)
toronto_neighbors = toronto_neighbors.dropna()
toronto_neighborhoods =toronto_neighbors.drop(['Postal Code'], axis=1)
toronto_neighborhoods

,PostalCode,Borough,Neighborhood,Latitude,Longitude
1,M3A,North York,Parkwoods,43.784535,-79.160497
2,M4A,North York,Victoria Village,43.763573,-79.188711
3,M5A,Downtown Toronto,Regent Park / Harbourfront,43.770992,-79.216917
4,M6A,North York,Lawrence Manor / Lawrence Heights,43.773136,-79.239476
5,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.744734,-79.239476
...,...,...,...,...,...
97,M9N,York,Weston,43.724766,-79.532242
98,M1P,Scarborough,Dorset Park / Wexford Heights / Scarborough To...,43.706876,-79.518188
99,M2P,North York,York Mills West,43.696319,-79.532242
101,M4P,Central Toronto,Davisville North,43.739416,-79.588437


In [17]:
toronto_neighborhoods.shape

(69, 5)

## Import Additional Libraries for Analysis

In [9]:
import json # library to handle JSON files


from geopy.geocoders import Nominatim 
GeoLocator = Nominatim(user_agent='My-IBMNotebook')# convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


import folium # map rendering library
print('Libraries imported.')

Libraries imported.


## Get the Coiordinates of Toronto

In [10]:
address = 'Toronto, Ontario Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto Canada are {}, {}.'.format(latitude, longitude))

C:\Users\markrussel.miranda\Documents\Documents\DUCMENTS\STUDIES\WPy64-3741\python-3.7.4.amd64\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Toronto Canada are 43.6534817, -79.3839347.


## Create a map of Torontovwith neighborhoods

In [18]:
# create map of Totonto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_neighborhoods['Latitude'], toronto_neighborhoods['Longitude'], toronto_neighborhoods['Borough'], toronto_neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#87cefa',
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto)

map_toronto

In [22]:
toronto_data = toronto_neighborhoods[toronto_neighborhoods['Borough'].str.contains("Toronto")].reset_index(drop=True)
print(toronto_data.shape)
toronto_data.head()

(22, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Regent Park / Harbourfront,43.770992,-79.216917
1,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.744734,-79.239476
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.794200,-79.262029
3,M5C,Downtown Toronto,St. James Town,43.789053,-79.408493
4,M4E,East Toronto,The Beaches,43.767980,-79.487262


In [23]:
# Foursquare API
CLIENT_ID = 'DPBYY4JUY3DU20ALPSUV4ONY2K1GOJJKJ1NIHBB32XEMOVYY' # Put Your Client Id
CLIENT_SECRET = '1MV443TYEP4HUO0WDUW5NQ5W10L2Y4G05NWG11WIR3NUGC5B' # Put You Client Secret 
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: Hidden')
print('CLIENT_SECRET: Hidden')

Your credentails:
CLIENT_ID: Hidden
CLIENT_SECRET: Hidden


## 1. Exploring Toronto Neighborhood

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([( name, lat, lng, v['venue']['name'], v['venue']['location']['lat'], v['venue']['location']['lng'], v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    
    return(nearby_venues)

In [25]:
df = toronto_data
toronto_venues = getNearbyVenues(names=df['Neighborhood'], latitudes=df['Latitude'],longitudes=df['Longitude'])

Regent Park / Harbourfront
Queen's Park / Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond / Adelaide / King
Dufferin / Dovercourt Village
Harbourfront East / Union Station / Toronto Islands
Little Portugal / Trinity
The Danforth West / Riverdale
Toronto Dominion Centre / Design Exchange
Brockton / Parkdale Village / Exhibition Place
India Bazaar / The Beaches West
Commerce Court / Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West


In [26]:
print(toronto_venues.shape)
toronto_venues.head()

(265, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Regent Park / Harbourfront,43.770992,-79.216917,Starbucks,43.770037,-79.221156,Coffee Shop
1,Regent Park / Harbourfront,43.770992,-79.216917,Tim Hortons,43.770827,-79.223078,Coffee Shop
2,Regent Park / Harbourfront,43.770992,-79.216917,Korean Grill House,43.770812,-79.214502,Korean Restaurant
3,Queen's Park / Ontario Provincial Government,43.744734,-79.239476,McCowan Park,43.745089,-79.239336,Playground
4,Queen's Park / Ontario Provincial Government,43.744734,-79.239476,Helen's Jewels By Park Lane,43.741801,-79.241916,Jewelry Store


In [27]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 120 uniques categories.



## 2. Analyze Each Borough


In [29]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,...,Tea Room,Tennis Court,Theater,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
toronto_onehot.shape

(265, 120)

In [31]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,...,Tea Room,Tennis Court,Theater,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Berczy Park,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
1,Brockton / Parkdale Village / Exhibition Place,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.033333,0.000000,0.066667,0.000000,0.033333,0.000000,0.00
2,Central Bay Street,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.000000,0.00
3,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
4,Commerce Court / Victoria Hotel,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.055556,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
5,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
6,Dufferin / Dovercourt Village,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,...,0.000000,0.0,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.00
7,Forest Hill North & West,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
8,"Garden District, Ryerson",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
9,Harbourfront East / Union Station / Toronto Is...,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.033333,0.0,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.00


In [32]:
toronto_grouped.shape

(21, 120)

In [33]:
num_top_venues = 5
for neigh in toronto_grouped['Neighborhood']:
    print("----"+neigh+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == neigh].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
          venue  freq
0       Airport  0.33
1      Bus Stop  0.33
2          Park  0.33
3  Home Service  0.00
4        Market  0.00


----Brockton / Parkdale Village / Exhibition Place----
                     venue  freq
0                     Café  0.10
1       Mexican Restaurant  0.07
2    Vietnamese Restaurant  0.07
3  Comfort Food Restaurant  0.03
4     Caribbean Restaurant  0.03


----Central Bay Street----
                  venue  freq
0     Indian Restaurant  0.10
1           Yoga Studio  0.05
2          Burger Joint  0.05
3        Discount Store  0.05
4  Fast Food Restaurant  0.05


----Christie----
                  venue  freq
0                  Park  0.67
1     Convenience Store  0.33
2  Gym / Fitness Center  0.00
3       Organic Grocery  0.00
4          Noodle House  0.00


----Commerce Court / Victoria Hotel----
                venue  freq
0       Grocery Store  0.22
1                Café  0.17
2                Park  0.11
3  Italian Restaurant  0.06
4  

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [35]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.shape

(21, 11)

## 3. Clustering Neighborhoods

In [36]:
from sklearn.cluster import KMeans
import sklearn.cluster.k_means_
km = KMeans(n_clusters=3, init='k-means++', max_iter=100, n_init=1, 
  verbose=True)

In [37]:
kclusters = 10
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=1).fit(toronto_grouped_clustering)
print(kmeans.labels_[0:10])
print(len(kmeans.labels_))

[9 2 2 3 2 2 2 4 8 2]
21


In [38]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Regent Park / Harbourfront,43.770992,-79.216917
1,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.744734,-79.239476
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.794200,-79.262029
3,M5C,Downtown Toronto,St. James Town,43.789053,-79.408493
4,M4E,East Toronto,The Beaches,43.767980,-79.487262


In [39]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = df
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Regent Park / Harbourfront,43.770992,-79.216917,5.0,Coffee Shop,Korean Restaurant,Drugstore,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Cuban Restaurant
1,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.744734,-79.239476,6.0,Playground,Cosmetics Shop,Jewelry Store,Women's Store,Donut Shop,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.794200,-79.262029,8.0,Latin American Restaurant,Clothing Store,Lounge,Skating Rink,Breakfast Spot,College Gym,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store
3,M5C,Downtown Toronto,St. James Town,43.789053,-79.408493,0.0,Home Service,Women's Store,Drugstore,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Cuban Restaurant
4,M4E,East Toronto,The Beaches,43.767980,-79.487262,1.0,Coffee Shop,Miscellaneous Shop,Caribbean Restaurant,Metro Station,Massage Studio,Bar,Women's Store,Dog Run,Comic Shop,Concert Hall


## Finally, let's visualize the resulting clusters

In [40]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'],kmeans.labels_):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon], radius=5, popup=label, color=rainbow[cluster-1], fill=True, fill_color=rainbow[cluster-1], fill_opacity=0.7).add_to(map_clusters)
map_clusters

# That's All Folks